In [1]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

2025-04-22 16:49:45.534035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745340585.790380      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745340585.863013      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import collections

import numpy as np
from keras.models import Sequential

from keras.utils import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [3]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.7 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64",

In [4]:
from datasets import load_dataset
squad_dataset = load_dataset("squad")

README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
squad_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})

In [6]:
# Shuffle and select random samples
squad_dataset = {
    "train": squad_dataset["train"].shuffle(seed=42).select(range(10000)),
    "validation": squad_dataset["validation"].shuffle(seed=42).select(range(2000)),
}

In [7]:
squad_dataset

{'train': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 10000
 }),
 'validation': Dataset({
     features: ['id', 'title', 'context', 'question', 'answers'],
     num_rows: 2000
 })}

In [8]:
# flatten all answer texts and compute max length
max_len = max(
    len(ans.split())
    for example in squad_dataset["train"]
    for ans in example["answers"]["text"]
)

print("Max answer length (words) in train set:", max_len)

Max answer length (words) in train set: 29


In [9]:
squad_dataset["train"][2]

{'id': '5727e2483acd2414000deef0',
 'title': 'Rule_of_law',
 'context': 'One important aspect of the rule-of-law initiatives is the study and analysis of the rule of law’s impact on economic development. The rule-of-law movement cannot be fully successful in transitional and developing countries without an answer to the question: does the rule of law matter for economic development or not? Constitutional economics is the study of the compatibility of economic and financial decisions within existing constitutional law frameworks, and such a framework includes government spending on the judiciary, which, in many transitional and developing countries, is completely controlled by the executive. It is useful to distinguish between the two methods of corruption of the judiciary: corruption by the executive branch, in contrast to corruption by private actors.',
 'question': 'In developing countries, who makes most of the spending decisions?',
 'answers': {'text': ['the executive'], 'answer_st

In [10]:
# 2) EXTRACT RAW TEXT LISTS
# ------------------------------------------------
def extract_texts(split):
    contexts  = [ex["context"]  for ex in split]
    questions = [ex["question"] for ex in split]
    answers    = [ex["answers"]["text"][0] for ex in split]  # first (gold) answer
    return contexts, questions, answers

train_ctxs, train_qs, train_as  = extract_texts(squad_dataset["train"])
val_ctxs,   val_qs,   val_as    = extract_texts(squad_dataset["validation"])

In [11]:
# 3) PREPARE DECODER INPUT/OUTPUT WITH <start>/<end> TOKENS
# ------------------------------------------------
train_in  = [f"<start> {a}" for a in train_as]
train_out = [f"{a} <end>" for a in train_as]
val_in    = [f"<start> {a}" for a in val_as]
val_out   = [f"{a} <end>" for a in val_as]

In [12]:
# 4) TOKENIZER SETUP
# ------------------------------------------------

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

all_texts = train_ctxs + train_qs + train_in + train_out \
          + val_ctxs   + val_qs   + val_in   + val_out

VOCAB_SIZE  = 20000
custom_filters = '!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n'  # note: removed '<' and '>'

tokenizer = Tokenizer(
    num_words=VOCAB_SIZE,
    oov_token="<UNK>",
    filters=custom_filters
)

tokenizer.fit_on_texts(all_texts)

In [13]:
tokenizer.word_index["<start>"]

12

In [14]:
tokenizer.word_index["<end>"]

13

In [15]:
# 5) ENCODE & PAD SEQUENCES

def encode_and_pad(texts, maxlen):
    seqs = tokenizer.texts_to_sequences(texts)
    return pad_sequences(seqs, maxlen=maxlen, padding="post")

MAX_CTX_LEN = 300
MAX_Q_LEN   = 30
MAX_ANS_LEN = 30

X_ctx_train = encode_and_pad(train_ctxs, MAX_CTX_LEN)
X_q_train   = encode_and_pad(train_qs,   MAX_Q_LEN)
X_ans_in_tr = encode_and_pad(train_in,    MAX_ANS_LEN)
y_ans_tr    = encode_and_pad(train_out,   MAX_ANS_LEN)

X_ctx_val   = encode_and_pad(val_ctxs,    MAX_CTX_LEN)
X_q_val     = encode_and_pad(val_qs,      MAX_Q_LEN)
X_ans_in_val= encode_and_pad(val_in,      MAX_ANS_LEN)
y_ans_val   = encode_and_pad(val_out,     MAX_ANS_LEN)

# expand last dim for sparse loss
y_ans_train = y_ans_tr.reshape(*y_ans_tr.shape, 1)
y_ans_valid = y_ans_val.reshape(*y_ans_val.shape, 1)

In [16]:
X_ans_in_val[0]

array([  12, 8078,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

## LSTM (Encoder-Decoder, No Attention Mechanism)
### Not used as the accuracy was very low < 5%

In [ ]:
# 6) BUILD ENCODER–DECODER
# ------------------------------------------------

import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Concatenate, AdditiveAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

EMBED_DIM = 128
ENC_UNITS = 64
DEC_UNITS = 64

# -- Context Encoder
ctx_in  = Input(shape=(MAX_CTX_LEN,), name="context_input")
ctx_emb = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(ctx_in)
_, st_h_c, st_c_c = LSTM(ENC_UNITS, return_state=True, name="ctx_encoder")(ctx_emb)

# -- Question Encoder
q_in   = Input(shape=(MAX_Q_LEN,), name="question_input")
q_emb  = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(q_in)
_, st_h_q, st_c_q = LSTM(ENC_UNITS, return_state=True, name="q_encoder")(q_emb)

# -- Merge states
merged_h = Concatenate()([st_h_c, st_h_q])
merged_c = Concatenate()([st_c_c, st_c_q])
proj_h   = Dense(DEC_UNITS, activation="tanh")(merged_h)
proj_c   = Dense(DEC_UNITS, activation="tanh")(merged_c)

# -- Decoder
ans_in   = Input(shape=(MAX_ANS_LEN,), name="answer_input")
ans_emb  = Embedding(VOCAB_SIZE, EMBED_DIM, mask_zero=True)(ans_in)
dec_lstm = LSTM(DEC_UNITS, return_sequences=True, return_state=True, name="decoder_lstm")
dec_outs, _, _ = dec_lstm(ans_emb, initial_state=[proj_h, proj_c])
dec_dense = Dense(VOCAB_SIZE, activation="softmax", name="decoder_output")
outputs   = dec_dense(dec_outs)

model = Model(
    inputs=[ctx_in, q_in, ans_in],
    outputs=outputs
)
model.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

In [ ]:
# 7) TRAIN
# ------------------------------------------------
model.fit(
    x   = [X_ctx_train, X_q_train, X_ans_in_tr],
    y   = y_ans_train,
    batch_size    = 64,
    epochs        = 10,
    validation_data = ([X_ctx_val, X_q_val, X_ans_in_val], y_ans_valid)
)


## LSTM (Encoder-Decoder, With Attention Mechanism)
### Not used as the accuracy was very low < 5%

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Concatenate, AdditiveAttention
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# ─── HYPERPARAMETERS ──────────────────────────────────────────────
EMBED_DIM     = 128
ENC_UNITS     = 256
DEC_UNITS     = 256
MAX_CTX_LEN   = 300
MAX_Q_LEN     = 30
MAX_ANS_LEN   = 30
LEARNING_RATE = 1e-4

# ─── INPUTS & EMBEDDINGS ──────────────────────────────────────────
ctx_input = Input(shape=(MAX_CTX_LEN,), name="context_input")
q_input   = Input(shape=(MAX_Q_LEN,),   name="question_input")
ans_input = Input(shape=(MAX_ANS_LEN,), name="answer_input")

# notice mask_zero=False → no masking
ctx_emb = Embedding(VOCAB_SIZE, EMBED_DIM, name="ctx_embedding")(ctx_input)
q_emb   = Embedding(VOCAB_SIZE, EMBED_DIM, name="q_embedding")(q_input)
ans_emb = Embedding(VOCAB_SIZE, EMBED_DIM, name="ans_embedding")(ans_input)

# ─── ENCODERS ─────────────────────────────────────────────────────
# Context encoder returns full sequence + final states
ctx_seq, ctx_h, ctx_c = LSTM(
    ENC_UNITS, return_sequences=True, return_state=True, name="ctx_encoder"
)(ctx_emb)

# Question encoder returns only final states
_, q_h, q_c = LSTM(
    ENC_UNITS, return_sequences=False, return_state=True, name="q_encoder"
)(q_emb)

# ─── STATE FUSION ────────────────────────────────────────────────
merged_h = Concatenate(name="merge_h")([ctx_h, q_h])
merged_c = Concatenate(name="merge_c")([ctx_c, q_c])
proj_h   = Dense(DEC_UNITS, activation="tanh", name="proj_h")(merged_h)
proj_c   = Dense(DEC_UNITS, activation="tanh", name="proj_c")(merged_c)

# ─── DECODER + ADDITIVE ATTENTION ─────────────────────────────────
dec_seq, _, _ = LSTM(
    DEC_UNITS,
    return_sequences=True,
    return_state=True,
    name="decoder_lstm"
)(ans_emb, initial_state=[proj_h, proj_c])

# use AdditiveAttention instead of Attention
attn_out = AdditiveAttention(name="attention")([dec_seq, ctx_seq])

dec_concat = Concatenate(name="decoder_concat")([dec_seq, attn_out])
decoder_outputs = Dense(
    VOCAB_SIZE, activation="softmax", name="decoder_output"
)(dec_concat)

# ─── MODEL & COMPILATION ────────────────────────────────────────
model = Model(
    inputs=[ctx_input, q_input, ans_input],
    outputs=decoder_outputs,
    name="QA_EncoderDecoder_with_AdditiveAttention"
)
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


In [ ]:
# 7) TRAINING THE MODEL

mask = (y_ans_train != 0).astype("float32")  # 1 for real tokens, 0 for padding

history = model.fit(
    x=[X_ctx_train, X_q_train, X_ans_in_tr],
    y=y_ans_train,
    sample_weight=mask,         # masks out padding positions
    batch_size=64,
    epochs=12,
    validation_data=(
        [X_ctx_val, X_q_val, X_ans_in_val],
        y_ans_valid
    )
)


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Assumes you have in scope:
#  - model                    : your trained encoder–decoder with attention
#  - tokenizer                : the fitted Tokenizer
#  - MAX_CTX_LEN, MAX_Q_LEN   : max lengths for context/question
#  - MAX_ANS_LEN              : max length for generated answer

def generate_answer(model, tokenizer, context, question,
                    max_ctx_len=MAX_CTX_LEN,
                    max_q_len=MAX_Q_LEN,
                    max_ans_len=MAX_ANS_LEN):
    # 1) Encode & pad context + question
    ctx_seq = tokenizer.texts_to_sequences([context])
    ctx_seq = pad_sequences(ctx_seq, maxlen=max_ctx_len, padding="post")

    q_seq   = tokenizer.texts_to_sequences([question])
    q_seq   = pad_sequences(q_seq,   maxlen=max_q_len,   padding="post")

    # 2) Prepare empty decoder input, seed with <start> token
    start_idx = tokenizer.word_index["<start>"]
    end_idx   = tokenizer.word_index["<end>"]
    ans_seq = np.zeros((1, max_ans_len), dtype="int32")
    ans_seq[0, 0] = start_idx

    # 3) Iteratively predict next token
    for i in range(1, max_ans_len):
        preds = model.predict([ctx_seq, q_seq, ans_seq], verbose=0)
        next_token = np.argmax(preds[0, i-1])
        print(f"Step {i}, next_token id = {next_token}")
        ans_seq[0, i] = next_token
        if next_token == end_idx:
            print("Hit <end>, stopping generation")
            break


    # 4) Convert token IDs back to words
    index_to_word = {idx: word for word, idx in tokenizer.word_index.items()}
    # skip padding (0) and <start>, stop at <end>
    output_tokens = []
    for idx in ans_seq[0]:
        if idx == 0 or idx == start_idx:
            continue
        if idx == end_idx:
            break
        output_tokens.append(index_to_word.get(idx, ""))

    return " ".join(output_tokens)

# ── Example Usage ───────────────────────────────────────────────────
context  = squad_dataset["validation"][2]["context"]
question = squad_dataset["validation"][2]["question"]
ground_truth_answer = squad_dataset["validation"][2]["answers"]["text"][0]

answer = generate_answer(model, tokenizer, context, question)
print("Context: ", context)
print("Question:", question)
print("Model Answer:  ", answer)
print("Ground Truth Answer:  ", ground_truth_answer)


## Transformers Encoder Decoder

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, Embedding, Dense, LayerNormalization, Dropout,
    MultiHeadAttention, Lambda, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [17]:
# Define temperature scaling layer
class TemperatureScaling(tf.keras.layers.Layer):
    def __init__(self, temperature=1.0, **kwargs):
        super().__init__(**kwargs)
        self.temperature = temperature

    def call(self, logits):
        return logits / self.temperature

In [ ]:
# Define masked loss and accuracy metrics
def masked_loss(y_true, y_pred, label_smoothing=0.1):
    # Squeeze the last dimension of y_true to match y_pred shape
    y_true = tf.squeeze(y_true, axis=-1)

    # Create mask to ignore padding tokens
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))
    mask = tf.cast(mask, dtype=tf.float32)

    # Convert y_true to integer type for one_hot encoding
    y_true = tf.cast(y_true, tf.int32)

    # Convert y_true to one-hot with label smoothing
    num_classes = tf.shape(y_pred)[-1]
    y_true = tf.one_hot(y_true, depth=num_classes, dtype=y_pred.dtype)

    # Apply label smoothing
    y_true = y_true * (1.0 - label_smoothing) + (
        label_smoothing / tf.cast(num_classes, y_pred.dtype)
    )

    # Calculate cross entropy loss
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=False)

    # Apply mask and return mean loss over non-padding tokens
    loss = loss * mask
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

In [ ]:

def masked_accuracy(y_true, y_pred):
    # Squeeze the last dimension of y_true to match y_pred shape
    y_true = tf.squeeze(y_true, axis=-1)

    # Create mask to ignore padding tokens
    mask = tf.math.logical_not(tf.math.equal(y_true, 0))

    # Get predicted tokens
    predictions = tf.argmax(y_pred, axis=-1)
    predictions = tf.cast(predictions, tf.int32)
    y_true = tf.cast(y_true, tf.int32)

    # Calculate accuracy only on non-padding tokens
    accuracies = tf.equal(y_true, predictions)
    accuracies = tf.math.logical_and(mask, accuracies)

    # Convert to float and take mean
    accuracies = tf.cast(accuracies, dtype=tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    return tf.reduce_sum(accuracies) / tf.reduce_sum(mask)

In [20]:
# ─── HYPERPARAMETERS ──────────────────────────────────────────────
D_MODEL       = 128
NUM_LAYERS    = 3
NUM_HEADS     = 8
DFF           = 512
DROPOUT_RATE  = 0.1
MAX_CTX_LEN   = 300
MAX_Q_LEN     = 30
MAX_SRC_LEN   = MAX_CTX_LEN + MAX_Q_LEN
MAX_ANS_LEN   = 30
LEARNING_RATE = 1e-4
CLIP_NORM = 1.0 # Add gradient clipping to prevent exploding gradients

# ─── POSITIONAL ENCODING ───────────────────────────────────────────
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, maxlen, d_model, **kwargs):
        super().__init__(**kwargs)
        pos = np.arange(maxlen)[:, None]
        i   = np.arange(d_model)[None, :]
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        angle_rads  = pos * angle_rates
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
        self.pos_encoding = tf.cast(angle_rads[None, ...], tf.float32)

    def call(self, x):
        return x + self.pos_encoding[:, :tf.shape(x)[1], :]

# ─── TRANSFORMER LAYERS ────────────────────────────────────────────
def transformer_encoder_layer(x, *, mask=None):
    attn_out = MultiHeadAttention(NUM_HEADS, key_dim=D_MODEL)(
        x, x, attention_mask=mask
    )
    attn_out = Dropout(DROPOUT_RATE)(attn_out)
    out1     = LayerNormalization(epsilon=1e-6)(x + attn_out)
    ffn      = Dense(DFF, activation="relu")(out1)
    ffn      = Dense(D_MODEL)(ffn)
    ffn      = Dropout(DROPOUT_RATE)(ffn)
    return LayerNormalization(epsilon=1e-6)(out1 + ffn)

def transformer_decoder_layer(x, enc_out, *, look_ahead_mask=None, padding_mask=None):
    attn1 = MultiHeadAttention(NUM_HEADS, key_dim=D_MODEL)(
        x, x, attention_mask=look_ahead_mask
    )
    attn1 = Dropout(DROPOUT_RATE)(attn1)
    out1  = LayerNormalization(epsilon=1e-6)(x + attn1)

    attn2 = MultiHeadAttention(NUM_HEADS, key_dim=D_MODEL)(
        out1, enc_out, enc_out, attention_mask=padding_mask
    )
    attn2 = Dropout(DROPOUT_RATE)(attn2)
    out2  = LayerNormalization(epsilon=1e-6)(out1 + attn2)

    ffn   = Dense(DFF, activation="relu")(out2)
    ffn   = Dense(D_MODEL)(ffn)
    ffn   = Dropout(DROPOUT_RATE)(ffn)
    return LayerNormalization(epsilon=1e-6)(out2 + ffn)

# ─── INPUTS ───────────────────────────────────────────────────────
ctx_input = Input(shape=(MAX_CTX_LEN,), name="context_input")
q_input   = Input(shape=(MAX_Q_LEN,),   name="question_input")
ans_input = Input(shape=(MAX_ANS_LEN,), name="answer_input")

# ─── SOURCE CONCAT & EMBEDDINGS ───────────────────────────────────
src     = Concatenate(name="src_concat")([ctx_input, q_input])
embed   = Embedding(VOCAB_SIZE, D_MODEL, name="token_embedding")
src_emb = embed(src)
tgt_emb = embed(ans_input)

# ─── ADD POSITIONAL ENCODING ──────────────────────────────────────
src_emb = PositionalEncoding(MAX_SRC_LEN, D_MODEL, name="pos_enc_src")(src_emb)
tgt_emb = PositionalEncoding(MAX_ANS_LEN, D_MODEL, name="pos_enc_tgt")(tgt_emb)

# ─── MASKS VIA LAMBDA ─────────────────────────────────────────────
enc_padding_mask = Lambda(
    lambda seq: tf.cast(tf.equal(seq, 0), tf.float32)[:, None, None, :],
    name="enc_padding_mask"
)(src)

look_ahead_mask = Lambda(
    lambda x: 1 - tf.linalg.band_part(
        tf.ones((MAX_ANS_LEN, MAX_ANS_LEN)), -1, 0
    )[None, None, :, :],
    name="look_ahead_mask"
)(ans_input)

dec_padding_mask = enc_padding_mask

# ─── ENCODER STACK ────────────────────────────────────────────────
enc_out = src_emb
for i in range(NUM_LAYERS):
    enc_out = transformer_encoder_layer(
        enc_out, mask=enc_padding_mask
    )

# ─── DECODER STACK ────────────────────────────────────────────────
dec_out = tgt_emb
for i in range(NUM_LAYERS):
    dec_out = transformer_decoder_layer(
        dec_out,
        enc_out,
        look_ahead_mask=look_ahead_mask,
        padding_mask=dec_padding_mask
    )

# ─── FINAL PROJECTION ─────────────────────────────────────────────
logits = Dense(VOCAB_SIZE, name="decoder_output")(dec_out)
logits = TemperatureScaling(temperature=0.7)(logits)
logits = tf.keras.layers.Activation("softmax")(logits)


# ─── MODEL & COMPILATION ────────────────────────────────────────
model = Model(
    inputs=[ctx_input, q_input, ans_input],
    outputs=logits,
    name="Transformer_QA_Seq2Seq"
)

optimizer = Adam(
    learning_rate=LEARNING_RATE,
    beta_1=0.9,
    beta_2=0.98,
    epsilon=1e-9,
    clipnorm=CLIP_NORM,
)

model.compile(
    optimizer=optimizer,
    loss=masked_loss,
    metrics=[masked_accuracy],
)

model.summary()

Model: "Transformer_QA_Seq2Seq"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ context_input             │ (None, 300)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ question_input            │ (None, 30)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ answer_input (InputLayer) │ (None, 30)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ src_concat (Concatenate)  │ (None, 330)            │              0 │ context_input[0][0],   │
│                           │                        │                │ question_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ token_embedding           │ (None, 30, 128)        │      2,560,000 │ src_concat[0][0],      │
│ (Embedding)               │                        │                │ answer_input[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pos_enc_src               │ (None, 330, 128)       │              0 │ token_embedding[0][0]  │
│ (PositionalEncoding)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ enc_padding_mask (Lambda) │ (None, 1, 1, 330)      │              0 │ src_concat[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_9    │ (None, 330, 128)       │        527,488 │ pos_enc_src[0][0],     │
│ (MultiHeadAttention)      │                        │                │ pos_enc_src[0][0],     │
│                           │                        │                │ enc_padding_mask[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_25 (Dropout)      │ (None, 330, 128)       │              0 │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_15 (Add)              │ (None, 330, 128)       │              0 │ pos_enc_src[0][0],     │
│                           │                        │                │ dropout_25[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_15    │ (None, 330, 128)       │            256 │ add_15[0][0]           │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 330, 512)       │         66,048 │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 330, 128)       │         65,664 │ dense_12[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_26 (Dropout)      │ (None, 330, 128)       │              0 │ dense_13[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_16 (Add)              │ (None, 330, 128)       │              0 │ layer_normalization_1… │
│                      

 Total params: 10,681,504 (40.75 MB)

 Trainable params: 10,681,504 (40.75 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
history = model.fit(
    x=[X_ctx_train, X_q_train, X_ans_in_tr],
    y=y_ans_train,
    batch_size=64,
    epochs=30,
    validation_data=(
        [X_ctx_val, X_q_val, X_ans_in_val],
        y_ans_valid
    )
)

Epoch 1/30


I0000 00:00:1745340743.966918     113 service.cc:148] XLA service 0x7ec4000234c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745340743.967769     113 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745340743.967788     113 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745340748.319395     113 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1745340768.772024     113 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


157/157 ━━━━━━━━━━━━━━━━━━━━ 143s 541ms/step - loss: 9.1142 - masked_accuracy: 0.2171 - val_loss: 7.7173 - val_masked_accuracy: 0.2412
Epoch 2/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 66s 422ms/step - loss: 7.2036 - masked_accuracy: 0.2376 - val_loss: 6.9142 - val_masked_accuracy: 0.2619
Epoch 3/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 64s 410ms/step - loss: 6.4614 - masked_accuracy: 0.2765 - val_loss: 6.5347 - val_masked_accuracy: 0.3045
Epoch 4/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 65s 417ms/step - loss: 6.0466 - masked_accuracy: 0.3176 - val_loss: 6.2968 - val_masked_accuracy: 0.3266
Epoch 5/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 65s 413ms/step - loss: 5.7872 - masked_accuracy: 0.3429 - val_loss: 6.1289 - val_masked_accuracy: 0.3593
Epoch 6/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 65s 412ms/step - loss: 5.5724 - masked_accuracy: 0.3700 - val_loss: 5.9936 - val_masked_accuracy: 0.3781
Epoch 7/30
157/157 ━━━━━━━━━━━━━━━━━━━━ 65s 413ms/step - loss: 5.3749 - masked_accuracy: 0.3877 - val_loss: 5.8802 - val_masked_accuracy: 0.

In [26]:
def generate_answer_transformer(
    model,
    tokenizer,
    context,
    question,
    max_ctx_len=MAX_CTX_LEN,
    max_q_len=MAX_Q_LEN,
    max_ans_len=MAX_ANS_LEN,
    temperature=0.6,
    top_k=20,
    top_p=0.9,
    min_length=3,
):
    # 1) Encode & pad context + question
    ctx_seq = tokenizer.texts_to_sequences([context])
    ctx_seq = pad_sequences(ctx_seq, maxlen=max_ctx_len, padding="post")

    q_seq = tokenizer.texts_to_sequences([question])
    q_seq = pad_sequences(q_seq, maxlen=max_q_len, padding="post")

    # 2) Prepare decoder input seeded with <start>
    start_idx = tokenizer.word_index["<start>"]
    end_idx = tokenizer.word_index["<end>"]
    pad_idx = 0
    ans_seq = np.zeros((1, max_ans_len), dtype="int32")
    ans_seq[0, 0] = start_idx

    # Get reverse word index for debugging
    idx2word = {v: k for k, v in tokenizer.word_index.items()}
    generated_tokens = []

    def nucleus_sampling(logits, temperature, top_k, top_p):
        # Apply temperature
        logits = logits / temperature

        # Remove low probability tokens
        if top_k > 0:
            top_k_logits, top_k_indices = tf.math.top_k(
                logits, k=min(top_k, logits.shape[-1])
            )
        else:
            top_k_logits, top_k_indices = logits, tf.range(logits.shape[-1])

        # Apply softmax to convert to probabilities
        probs = tf.nn.softmax(top_k_logits)

        # Nucleus (top-p) sampling
        if top_p < 1.0:
            sorted_probs = tf.sort(probs, direction="DESCENDING")
            cumsum_probs = tf.cumsum(sorted_probs)
            cutoff = tf.where(cumsum_probs > top_p)[0][0]
            probs = tf.where(probs < sorted_probs[cutoff], tf.zeros_like(probs), probs)
            # Renormalize probabilities
            probs = probs / tf.reduce_sum(probs)

        # Sample from the filtered distribution
        sample_idx = tf.random.categorical(tf.math.log(probs[None, :]), num_samples=1)[
            0, 0
        ]
        return top_k_indices[sample_idx], probs[sample_idx]

    # 3) Generate with nucleus sampling
    for i in range(1, max_ans_len):
        # Get model predictions
        preds = model.predict([ctx_seq, q_seq, ans_seq], verbose=0)
        next_token_logits = preds[0, i - 1]

        # Mask padding token
        next_token_logits = tf.where(
            tf.range(len(next_token_logits)) == pad_idx,
            tf.fill(next_token_logits.shape, -1e9),
            next_token_logits,
        )

        # Don't allow END token before min_length
        if i < min_length:
            next_token_logits = tf.where(
                tf.range(len(next_token_logits)) == end_idx,
                tf.fill(next_token_logits.shape, -1e9),
                next_token_logits,
            )

        # Apply nucleus sampling
        next_id, prob = nucleus_sampling(next_token_logits, temperature, top_k, top_p)

        # Print debugging info
        token = idx2word.get(next_id.numpy(), "<UNK>")
        print(f"Step {i}: Selected token '{token}' with probability {prob:.4f}")

        # Save the generated token
        generated_tokens.append(token)

        # Update sequence
        ans_seq[0, i] = next_id

        # Break if we generate END token
        if next_id == end_idx:
            break

    # Print full generation info
    print("\nGeneration summary:")
    print("Generated tokens:", " ".join(generated_tokens))
    print("Probabilities:", [f"{t}: {p:.4f}" for t, p in zip(generated_tokens, [prob])])

    # 4) Convert IDs back to words, skipping special tokens
    output = []
    for idx in ans_seq[0]:
        if idx == pad_idx or idx == start_idx:
            continue
        if idx == end_idx:
            break
        token = idx2word.get(idx, "")
        if token and not token.startswith("<"):
            output.append(token)

    return " ".join(output)


# ─── Test on one example ─────────────────────────────────────────────
print("\nTesting generation with new parameters...")
for index in range(3):  # Test on fewer examples
    example = squad_dataset["validation"][index]
    context = example["context"]
    question = example["question"]
    ground_truth = example["answers"]["text"][0]

    print("\n" + "=" * 80)
    print("Context:  ", context)
    print("\nQuestion: ", question)
    print(
        "\nPredicted:",
        generate_answer_transformer(
            model, tokenizer, context, question, temperature=0.6, top_k=20, top_p=0.9
        ),
    )
    print("Ground Truth:", ground_truth)
    print("=" * 80)


Testing generation with new parameters...

Context:   Private schooling in the United States has been debated by educators, lawmakers and parents, since the beginnings of compulsory education in Massachusetts in 1852. The Supreme Court precedent appears to favor educational choice, so long as states may set standards for educational accomplishment. Some of the most relevant Supreme Court case law on this is as follows: Runyon v. McCrary, 427 U.S. 160 (1976); Wisconsin v. Yoder, 406 U.S. 205 (1972); Pierce v. Society of Sisters, 268 U.S. 510 (1925); Meyer v. Nebraska, 262 U.S. 390 (1923).

Question:  In what year did Massachusetts first require children to be educated in schools?
Step 1: Selected token 'war' with probability 0.0556
Step 2: Selected token '000' with probability 0.0556
Step 3: Selected token '<end>' with probability 0.2189

Generation summary:
Generated tokens: war 000 <end>
Probabilities: ['war: 0.2189']

Predicted: war 000
Ground Truth: 1852

Context:   The chloroplast

## Plot Loss and accuracy

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import json

# Create figure with two subplots
plt.style.use("seaborn")
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 12))

# Plot Loss
epochs = range(1, len(history.history["loss"]) + 1)
ax1.plot(epochs, history.history["loss"], "b-", label="Training Loss")
ax1.plot(epochs, history.history["val_loss"], "r-", label="Validation Loss")
ax1.set_title("Model Loss Over Time")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.legend()
ax1.grid(True)

# Add final values annotation
ax1.annotate(
    f"Final Train Loss: {history.history['loss'][-1]:.4f}",
    xy=(epochs[-1], history.history["loss"][-1]),
    xytext=(epochs[-1] - 2, history.history["loss"][-1] + 0.2),
    arrowprops=dict(facecolor="black", shrink=0.05),
)
ax1.annotate(
    f"Final Val Loss: {history.history['val_loss'][-1]:.4f}",
    xy=(epochs[-1], history.history["val_loss"][-1]),
    xytext=(epochs[-1] - 2, history.history["val_loss"][-1] + 0.2),
    arrowprops=dict(facecolor="black", shrink=0.05),
)

# Plot Accuracy
ax2.plot(epochs, history.history["masked_accuracy"], "b-", label="Training Accuracy")
ax2.plot(epochs, history.history["val_masked_accuracy"], "r-", label="Validation Accuracy")
ax2.set_title("Model Accuracy Over Time")
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Masked Accuracy")
ax2.legend()
ax2.grid(True)

# Add final values annotation
ax2.annotate(
    f"Final Train Acc: {history.history['masked_accuracy'][-1]:.4f}",
    xy=(epochs[-1], history.history["masked_accuracy"][-1]),
    xytext=(epochs[-1] - 2, history.history["masked_accuracy"][-1] - 0.05),
    arrowprops=dict(facecolor="black", shrink=0.05),
)
ax2.annotate(
    f"Final Val Acc: {history.history['val_masked_accuracy'][-1]:.4f}",
    xy=(epochs[-1], history.history["val_masked_accuracy"][-1]),
    xytext=(epochs[-1] - 2, history.history["val_masked_accuracy"][-1] - 0.05),
    arrowprops=dict(facecolor="black", shrink=0.05),
)

# Add analysis text
plt.figtext(
    0.02,
    0.02,
    """
Analysis:
- Training converges but shows signs of overfitting (validation loss increases)
- Final validation loss (4.64) is ~31% higher than training loss (3.52)
- Accuracy improvement slows after epoch 20
- Possible solutions: early stopping, stronger regularization, or learning rate decay
""",
    fontsize=10,
    bbox=dict(facecolor="white", alpha=0.8),
)

# Adjust layout and save
plt.tight_layout()
plt.savefig("training_history.png", bbox_inches="tight", dpi=300)
plt.close()

print("Training history plots have been saved to 'training_history.png'")


/tmp/ipykernel_31/54687702.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


Training history plots have been saved to 'training_history.png'
